## Single Vending Machine Optimization Problem

### Problem Description

The Vending machine optimisation is closely related to assignment problem, which is concerned with the optimal placement of products (from a set of products) in a machine having specific capacity in order to maximize the minimum days of supply (dos) for each machine. 

#### About Vending machine [Input file - machines.csv]

There are a few(12) vending machines at different locations, each machine has unique machine id, machine_type, list of the product types that the machine can occupy, and supply per day(spd) of each product associated to the machine id.

#### About Products[Input file - items.csv]

There are total 5 types of products with unique product ids eg. [1:soda,2:cola,3:tropics,4:cherry,5:cola can]
and each product has a volumne(pack) of container eg. 16.9oz bottle,12oz can.

#### Product Machine[Input file - capacity_matrix.csv]

The volumne of the product container(pack) and machine_type decides how much space that product has in the vending machine.

### Assumptions

* Each column and occupy only one type of product, a column can't be shared betwwen two or mode products
* Once the product ids assigned to the column, it uses it's total capacity.
* One prodict can be placed in multiple columns but one column can't occupy more than one products.





### Solution Approach
A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.
We now present a BIP formulation for the Vending Mcachine optimisation problem.### Check twice

## Model Formulation

### Sets 

machine $m \in M$: Set of Machines.{1, 2, 3,.....12}

machines_types $i \in I$: Set Machine Types, {closed_front_1, closed_front_2, closed_front_3, closed_front_4, closed_front_5, closed_front_6}.

columns $c \in C$: Set of Columns in a machine.{col1, col2, col3, col4, col5, col6,col7, col8, col9, col10, col11, col12}

products $p \in P$: Set of products {111, 222, 333, 444, 555}

volumns $v \in V$: Set of volumns {12oz,16.9oz}




### Parameters

$cap_{i,v,c} \in \mathbb{R}^+$: Fixed capacity associated with machine type, volumn of product, and machine column for machine type $i \in I$ for volumn  $v \in V$ for column $c \in C$.

$spdm_{m,i,p} \in \mathbb{R}^+$: Supply per day for a machine $m \in M$ and type $i \in I$ of product $p \in P$.

$vol_{p} \in \mathbb{R}^+$: Volume of product  $p \in P$. 




### Decision Variables

$selectsingle_{m,i,c,p}\in \{0,1\}$: 1 If product p  is placed in collumn c of machine type i in machine m; 0 therwise that 





### Objective Function


- **Minimum days of supply**: Maximize the minimum days of supply for machine m :

\begin{equation}
\text{Min} \quad Z[p] = \sum_{(i,j) \in \text{Pairings}}\text{weight}_i \cdot \text{dist}_{i,j} \cdot \text{assign}_{i,j}
\tag{0}
\end{equation}

\begin{equation}
\text{Maximise} \quad \text{Min} y[p] = \sum_{m \in M} \sum_{i \in I}\sum_{c \in C} selectsingle_{m,i,c,p}*cap_{i,v,c}
\tag{0}
\end{equation}



\begin{equation}
\quad y[p]=\frac{\sum_{(m,i,c,p) \in \text{Pairings}} selectsingle_{m,i,c,p}*cap_{i,v,c}}{spdm_{m,i,p}} \text{  } \forall p \in \text{Products}
\tag{0}
\end{equation}

\begin{equation}
\text{Maximize} \quad z=\text{Min} \text{} y[p] \forall p \in \text{Products}
\end{equation}


### Constraints


- **One column one product constraint**. For each column  $c \in C$ ensure that it contains only one type of product.


\begin{equation}
\sum_{m \in M} \sum_{i \in I}\sum_{p \in P}{selectsingle_{m,i,c,p}} \leq 1 \forall c \in \text{Columns}
\end{equation}






In [14]:
pip install gurobipy==9.5.2

  Using cached gurobipy-9.5.2-cp38-cp38-macosx_10_9_x86_64.whl (7.4 MB)
  Attempting uninstall: gurobipy
    Found existing installation: gurobipy 10.0.0
    Uninstalling gurobipy-10.0.0:
      Successfully uninstalled gurobipy-10.0.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import multidict
from gurobipy import GRB
from gurobipy import min_

In [2]:
### Reading required files
capacity = pd.read_csv('capacity_matrix.csv')
items = pd.read_csv('items.csv')
machines = pd.read_csv('machines.csv')
machine_id = 11
v=machines.loc[machines['machine_id']==machine_id,'machine_type'].unique()
machine_type = v[0]
columns_shelf=[col for col in capacity.columns if 'col' in col]

In [3]:

## Handling Single Machine 
machines_single = machines[machines['machine_id']==machine_id]

## Adding column_name optimions column to the machines 
machines_single=machines_single.merge(pd.DataFrame(columns_shelf), how='cross')
machines_single.rename(columns = {0:'Column'}, inplace = True)

In [4]:
## Data Correction- Not needed for single machine 
correction = {'soda':111 , 'cola':222, 'tropics':333, 'cherry':444,'cola can':555}
machines_single=machines_single.replace({"item_id": correction})
machines_single['item_id'] =machines_single['item_id'].apply(pd.to_numeric)

In [5]:
#Sets-Single machine 
products = list(items['item_id'].unique())
machine = list(machines_single['machine_id'].unique())
machines_types = list(machines_single['machine_type'].unique())
columns = columns_shelf#list(columns_shelf[2:14])#list(range(1,13)) ## conver to auto
volumns = list(capacity['pack'].unique())


In [6]:
#Single machine -Parameters -1
cap={}
for j in machines_types: 
    for k in volumns:
        for col in columns_shelf:
          column_name = col #'col'+str(col)
          cap[j,k,column_name]=capacity.loc[(capacity['machine_type']==j) & (capacity['pack']==k),column_name].values[0]
            

## Single Parameters -2
#𝑠𝑝𝑑𝑚,𝑖,𝑝∈ℝ+ : Supply per day for a machine 𝑚∈𝑀 and type 𝑖∈𝐼 of product 𝑝∈𝑃.
    
spdm = {}
spdm={j:k for j in zip(machines_single['machine_id'],machines_single['machine_type'],machines_single['item_id']) for k in machines_single.loc[(machines_single['machine_id']==j[0]) & (machines_single['machine_type']==j[1]) & (machines_single['item_id']==j[2]),'spd']}


## Parameters -3
#𝑣𝑜𝑙𝑝,𝑣∈ℝ+ : Volume of product  𝑝∈𝑃  is  𝑣∈𝑉 .
volp ={p:v for p in items['item_id'] for v in items.loc[items['item_id']==p,'pack'] }

                  

In [24]:
pip show gurobipy

Name: gurobipy
Version: 9.5.2
Summary: Python interface to Gurobi
Home-page: https://www.gurobi.com
Author: Gurobi Optimization, LLC
Author-email: 
License: Proprietary
Location: /opt/anaconda3/envs/New_python/lib/python3.8/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
## Decision Variable -single machine 
select1 ={j:0 for j in zip(machines_single['machine_id'],machines_single['machine_type'],machines_single['item_id'],machines_single['Column'])}
combinations, scores = gp.multidict(select1)

In [8]:
##Single mahine - Start model  create decision variable -

model = gp.Model('VendingMachineOptimization_singlemachine')
select_single = model.addVars(combinations, vtype=GRB.BINARY, name='select2')  #(1,'closed_front_2', '555', 1):0/1
#select = model.addVars(combinations, obj=scores, name="prod_col")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-02


In [9]:
## Modification in above
#machine_type='closed_front_3'
#machine_id=9

#y = model.addVars(products,lb=0,ub=GRB.INFINITY, name="y")
y = model.addVars(products, lb=-GRB.INFINITY, name="y")

z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")
model.addConstrs((y[p] == gp.quicksum(select_single[machine_id,machine_type,p,c]*cap[machine_type,volp[int(p)],c] for c in columns_shelf)/spdm[machine_id,machine_type,p] 
                  for p in products),name="set_y")
model.addConstr(z == min_(y), name="min constraint")
model.setObjective(z, sense=GRB.MAXIMIZE)

In [10]:
## Single machine - Constraint2
model.addConstrs((select_single.sum(m,i,'*',c) <= 1
             for m,i,p,c in select_single.keys()),
            name="single_1_col_1prod") 


{(11, 'closed_front_5', 111, 'col1'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col2'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col3'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col4'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col5'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col6'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col7'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col8'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col9'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col10'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col11'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 111, 'col12'): <gurobi.Constr *Awaiting Model Update*>,
 (11, 'closed_front_5', 222, 'col1'): <gurobi.Con

In [123]:
## Single machine Minimum demand has to be satisfied - Constraint2
"""
model.addConstrs((select_single.sum(m,i,p,'*')*cap[i,volp[int(p)],c] >= spdm[m,i,p]
             for m,i,p,c in select_single.keys()),
            name="single_min_demand") """


'\nmodel.addConstrs((select_single.sum(m,i,p,\'*\')*cap[i,volp[int(p)],c] >= spdm[m,i,p]\n             for m,i,p,c in select_single.keys()),\n            name="single_min_demand") '

In [11]:
# Find the optimal solution
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 65 rows, 66 columns and 365 nonzeros
Model fingerprint: 0xb3e85836
Model has 1 general constraint
Variable types: 6 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 48 rows and 5 columns
Presolve time: 0.00s
Presolved: 17 rows, 61 columns, 125 nonzeros
Variable types: 1 continuous, 60 integer (60 binary)

Root relaxation: objective 3.872255e+00, 42 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.87226    0    7   -0.00000    3.87226      -     -    0s
H    0

In [12]:
### Post processing output values 
product_flow = pd.DataFrame(columns=["Machine_id", "Machine_type", "Product","Column","Values","Capacity"])
for arc in combinations:
    #if select_single[arc].x > 1e-6:
    product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]],"dos":select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]]/spdm[arc[0],arc[1],arc[2]],"spd":spdm[arc[0],arc[1],arc[2]]}, ignore_index=True)  
#product_flow.index=[''] * len(product_flow)
product_flow

/var/folders/1n/0zx6lq554296fqmshk5ysbvc0000gn/T/ipykernel_9350/1727328463.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]],"dos":select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]]/spdm[arc[0],arc[1],arc[2]],"spd":spdm[arc[0],arc[1],arc[2]]}, ignore_index=True)
/var/folders/1n/0zx6lq554296fqmshk5ysbvc0000gn/T/ipykernel_9350/1727328463.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[

,Machine_id,Machine_type,Product,Column,Values,Capacity,dos,spd
0,11,closed_front_5,111,col1,-0.0,-0.0,-0.000000,12.1
1,11,closed_front_5,111,col2,-0.0,-0.0,-0.000000,12.1
2,11,closed_front_5,111,col3,-0.0,-0.0,-0.000000,12.1
3,11,closed_front_5,111,col4,1.0,14.0,1.157025,12.1
4,11,closed_front_5,111,col5,1.0,14.0,1.157025,12.1
5,11,closed_front_5,111,col6,-0.0,-0.0,-0.000000,12.1
6,11,closed_front_5,111,col7,-0.0,-0.0,-0.000000,12.1
7,11,closed_front_5,111,col8,-0.0,-0.0,-0.000000,12.1
8,11,closed_front_5,111,col9,-0.0,-0.0,-0.000000,12.1
9,11,closed_front_5,111,col10,1.0,14.0,1.157025,12.1


In [15]:
### Post processing output values 
product_flow = pd.DataFrame(columns=["Machine_id", "Machine_type", "Product","Column","Values","Capacity"])
for arc in combinations:
 if select_single[arc].x > 1e-6:
    product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]],"dos":select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]]/spdm[arc[0],arc[1],arc[2]],"spd":spdm[arc[0],arc[1],arc[2]]}, ignore_index=True)  
#product_flow.index=[''] * len(product_flow)

/var/folders/1n/0zx6lq554296fqmshk5ysbvc0000gn/T/ipykernel_9350/3437203792.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]],"dos":select_single[arc].x*cap[arc[1],volp[int(arc[2])],arc[3]]/spdm[arc[0],arc[1],arc[2]],"spd":spdm[arc[0],arc[1],arc[2]]}, ignore_index=True)
/var/folders/1n/0zx6lq554296fqmshk5ysbvc0000gn/T/ipykernel_9350/3437203792.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  product_flow = product_flow.append({"Machine_id": arc[0], "Machine_type": arc[1],"Product": arc[2], "Column": arc[3],"Values": select_single[arc].x,"Capacity": select_single[arc].x*cap[arc[1],volp[int(arc[

In [16]:
column_k=product_flow[product_flow['Values']==1]

In [17]:
k=product_flow.drop(['Column','Values'], axis=1)

In [18]:
j=k.groupby(['Machine_id','Machine_type','Product','spd']).agg(sum)

In [19]:
j

Capacity       dos
Machine_id Machine_type   Product spd                     
11         closed_front_5 111     12.1      42.0  3.471074
                          222     9.8       42.0  4.285714
                          333     8.6       28.0  3.255814
                          444     4.7       24.0  5.106383
                          555     12.1      46.0  3.801653

In [172]:
j

Capacity       dos
Machine_id Machine_type   Product spd                     
9          closed_front_3 111     12.1      44.0  3.636364
                          222     9.8       32.0  3.265306
                          333     8.6       33.0  3.837209
                          444     4.7       18.0  3.829787
                          555     12.1      36.0  2.975207

In [20]:
product_flow[['Machine_id','Machine_type','Product','Column']]

,Machine_id,Machine_type,Product,Column
0,11,closed_front_5,111,col4
1,11,closed_front_5,111,col5
2,11,closed_front_5,111,col10
3,11,closed_front_5,222,col6
4,11,closed_front_5,222,col8
5,11,closed_front_5,222,col9
6,11,closed_front_5,333,col3
7,11,closed_front_5,333,col7
8,11,closed_front_5,444,col1
9,11,closed_front_5,444,col2


In [21]:
model.write('vending.lp')

In [177]:
model.getParamInfo('Heuristics')

('Heuristics', float, 0.05, 0.0, 1.0, 0.05)

In [178]:
model.ObjVal

2.975206611570248

In [179]:
model.NumVars#	model.NumConstrs	model.IterCount	model.NodeCount	model.Runtime

162

In [180]:
model.NumConstrs

140

In [181]:
model.IterCount

69.0

In [182]:
model.NodeCount

1.0

In [183]:
model.Runtime

0.03797316551208496

In [ ]:
6.111	300	220	1234	11	0.5